# Notebook to define ensemble for production, pgm level

Version developed for the UK FCDO

This notebook defines the ensemble used for production: selects a set of pre-trained models, retrieves and calibrates them, computes and stores the ensemble model predictions.

Models are stored in model storage and most of them specified in the notebook fat_pgm_constituentmodels

The notebook draws on the following .py script files in this repository:

Ensembling.py

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *
# Other packages
import pickle as pkl

In [3]:
# Packages from Predicting Fatalies repository

from HurdleRegression import *
import Ensembling
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

from FetchData import FetchData_pgm, RetrieveFromList

In [4]:
# Common parameters:

dev_id = 'Fatalities001'
run_id = 'Fatalities001'
EndOfHistory = 506

cm_calibration_calib = 'cm_ensemble_genetic_calib'
cm_calibration_test = 'cm_ensemble_genetic_test'

level = 'pgm'

depvar = "ln_ged_sb_dep"

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = '/Users/jim/Dropbox (ViEWS)/ViEWS/'
overleafpath = '/Users/jim/Dropbox (ViEWS)/Apps/Overleaf/ViEWS predicting fatalities/Tables/'

In [17]:
#dbpath = Mydropbox + '/Projects/PredictingFatalities/Predictions/'
#filename = dbpath + 'AllModels_pgm_' + dev_id + '.csv'

gitname = 'ModelList_pgm_wide_' + dev_id + '.csv'
ModelList_df = pd.read_csv(gitname)


#allmodels=pd.read_csv(filename).drop(['Unnamed: 0',],axis=1)

In [21]:
ModelList_df

,Unnamed: 0,modelname,algorithm,depvar,queryset,data_train,Algorithm_text,RunResult_calib,predstore_calib,RunResult_test,predstore_test,RunResult_future,predstore_future
0,0,fat_jed_hh_baseline_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_baseline,baseline,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_baseline_lgbm_calib,RunResult(training_date = 2022-05-16),pgm_fat_jed_hh_baseline_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_baseline_lgbm_f506
1,1,fat_jed_hh_conflictlong_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_conflictlong,conflictlong,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_lgbm_calib,RunResult(training_date = 2022-05-17),pgm_fat_jed_hh_conflictlong_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_lgbm_f506
2,2,fat_jed_hh_conflictlong_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,hh_fat_pgm_conflictlong,conflictlong,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_hurdle_lgbm_calib,RunResult(training_date = 2022-05-17),pgm_fat_jed_hh_conflictlong_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_hurdle_lgbm_f506
3,3,fat_jed_hh_drought_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,fat_escwa_drought_vulnerability_pgm,escwa_drought,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_hurdle_lgbm_calib,RunResult(training_date = 2022-05-17),pgm_fat_jed_hh_drought_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_hurdle_lgbm_f506
4,4,fat_jed_hh_drought_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,fat_escwa_drought_vulnerability_pgm,escwa_drought,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_drought_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_lgbm_f506
5,5,fat_jed_hh_natsoc_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,hh_fat_pgm_natsoc,natsoc,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_hurdle_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_natsoc_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_hurdle_lgbm_f506
6,6,fat_jed_hh_natsoc_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_natsoc,natsoc,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_natsoc_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_lgbm_f506
7,7,fat_jed_hh_broad_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,hh_fat_pgm_broad,broad,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_hurdle_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_broad_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_hurdle_lgbm_f506
8,8,fat_jed_hh_broad_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_broad,broad,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_broad_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_lgbm_f506
9,9,fat_jed_pv_conf_hist_pgm,"XGBRegressor(base_score=None, booster=None, co...",ln_ged_sb_dep,paola_fatalities_conflict_history,paola_conf_hist,"XGBRegressor(base_score=None, booster=None, co...",RunResult(training_date = 2022-03-18),pgm_fat_jed_pv_conf_hist_pgm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_pv_conf_hist_pgm_test,RunResult(trainin

In [31]:
# Defining the ensemble
# First item in dictionary is model name, second run id for development run

ModelsToRead = [
    'fat_jed_hh_baseline_lgbm','fat_devel_v7',
    'fat_jed_hh_conflictlong_lgbm','fat_devel_v7',
    'fat_jed_hh_conflictlong_hurdle_lgbm','fat_devel_v7',
    'fat_jed_hh_drought_hurdle_lgbm','fat_devel_v7',
    'fat_jed_hh_drought_lgbm','fat_devel_v7',    
    'fat_jed_hh_natsoc_hurdle_lgbm','fat_devel_v7',
    'fat_jed_hh_natsoc_lgbm','fat_devel_v7',
    'fat_jed_hh_broad_hurdle_lgbm','fat_devel_v7',
    'fat_jed_hh_broad_lgbm','fat_devel_v7',
    'fat_jed_pv_conf_hist_pgm','fat_devel_v7',
    'fat_tree_lags_d_1_d_2_hur_36','fat_devel_v7',
    'fat_sptime_dist_nu1_10_001_hur_36','fat_devel_v7',
]


#ModelsToRead = [
#    ['fat_jed_hh_baseline_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_conflictlong_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_conflictlong_hurdle_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_drought_hurdle_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_drought_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_natsoc_hurdle_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_natsoc_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_broad_hurdle_lgbm','fat_devel_v7'],
#    ['fat_jed_hh_broad_lgbm','fat_devel_v7'],
#    ['fat_jed_pv_conf_hist_pgm','fat_devel_v7'],
#    ['fat_tree_lags_d_1_d_2_hur_36','fat_devel_v7'],
#    ['fat_sptime_dist_nu1_10_001_hur_36','fat_devel_v7'],
#]

EnsembleMetaData = ModelList_df[ModelList_df['modelname'].isin(ModelsToRead)].copy()
ModelList = EnsembleMetaData.to_dict('records')

# Saving revised model list in data frame form

gitname = 'EnsembleMetaData_pgm_' + dev_id + '.csv'
EnsembleMetaData.to_csv(gitname)

#ModelList = []

#i = 0
#for item in ModelsToRead:

#    try:
#        modelname=ModelsToRead[i][0]
        
#        row=allmodels.loc[allmodels['modelname'] == modelname]
    
#        model = {
#            'modelname': ModelsToRead[i][0],
#            'depvar': row['depvar'].values[0],
#            'data_train': row['data_train'].values[0],
#            'queryset': row['queryset'].values[0],
#            'algorithm': row['algorithm'].values[0],
#            'Algorithm_text': row['Algorithm_text'].values[0],
#            }
    
#        ModelList.append(model)
#        print('added',modelname,'to model list')
    
#    except:
#        print('model',modelname,'not in csv file of trained models')
    
#    i = i + 1
#EnsembleMetaData = pd.DataFrame(ModelList)

In [23]:
EnsembleMetaData.head(12)

,Unnamed: 0,modelname,algorithm,depvar,queryset,data_train,Algorithm_text,RunResult_calib,predstore_calib,RunResult_test,predstore_test,RunResult_future,predstore_future
0,0,fat_jed_hh_baseline_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_baseline,baseline,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_baseline_lgbm_calib,RunResult(training_date = 2022-05-16),pgm_fat_jed_hh_baseline_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_baseline_lgbm_f506
1,1,fat_jed_hh_conflictlong_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_conflictlong,conflictlong,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_lgbm_calib,RunResult(training_date = 2022-05-17),pgm_fat_jed_hh_conflictlong_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_lgbm_f506
2,2,fat_jed_hh_conflictlong_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,hh_fat_pgm_conflictlong,conflictlong,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_hurdle_lgbm_calib,RunResult(training_date = 2022-05-17),pgm_fat_jed_hh_conflictlong_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_conflictlong_hurdle_lgbm_f506
3,3,fat_jed_hh_drought_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,fat_escwa_drought_vulnerability_pgm,escwa_drought,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_hurdle_lgbm_calib,RunResult(training_date = 2022-05-17),pgm_fat_jed_hh_drought_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_hurdle_lgbm_f506
4,4,fat_jed_hh_drought_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,fat_escwa_drought_vulnerability_pgm,escwa_drought,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_drought_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_drought_lgbm_f506
5,5,fat_jed_hh_natsoc_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,hh_fat_pgm_natsoc,natsoc,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_hurdle_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_natsoc_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_hurdle_lgbm_f506
6,6,fat_jed_hh_natsoc_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_natsoc,natsoc,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_natsoc_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_natsoc_lgbm_f506
7,7,fat_jed_hh_broad_hurdle_lgbm,"HurdleRegression(clf_name='LGBMClassifier', cl...",ln_ged_sb_dep,hh_fat_pgm_broad,broad,"HurdleRegression(clf_name='LGBMClassifier', cl...",RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_hurdle_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_broad_hurdle_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_hurdle_lgbm_f506
8,8,fat_jed_hh_broad_lgbm,LGBMRegressor(n_estimators=200),ln_ged_sb_dep,hh_fat_pgm_broad,broad,LGBMRegressor(n_estimators=200),RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_lgbm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_hh_broad_lgbm_test,RunResult(training_date = 2022-02-23),pgm_fat_jed_hh_broad_lgbm_f506
9,9,fat_jed_pv_conf_hist_pgm,"XGBRegressor(base_score=None, booster=None, co...",ln_ged_sb_dep,paola_fatalities_conflict_history,paola_conf_hist,"XGBRegressor(base_score=None, booster=None, co...",RunResult(training_date = 2022-03-18),pgm_fat_jed_pv_conf_hist_pgm_calib,RunResult(training_date = 2022-05-18),pgm_fat_jed_pv_conf_hist_pgm_test,RunResult(trainin

In [9]:
# Saving revised model list in data frame form
dbpath = Mydropbox + 'Projects/PredictingFatalities/Predictions/'
filename = dbpath + 'ModelMetaData_pgm_' + dev_id + '.csv'
EnsembleMetaData.to_csv(filename)

# Retrieve and calibrate predictions

In [24]:
# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model

ModelList = Ensembling.RetrieveStoredPredictions_pgm(ModelList, steps, EndOfHistory, run_id, level, True)

# we are not calibrating the *constituent* models at pgm

#ModelList = CalibratePredictions(ModelList, EndOfHistory, steps)

0 fat_jed_hh_baseline_lgbm
pr_45_pgm_fat_jed_hh_baseline_lgbm_calib.parquet


Exception during reset or similar
Traceback (most recent call last):
  File "/Users/jim/opt/anaconda3/envs/viewser/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/jim/opt/anaconda3/envs/viewser/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/Users/jim/opt/anaconda3/envs/viewser/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
psycopg2.DatabaseError: SSL SYSCALL error: Operation timed out



pr_45_pgm_fat_jed_hh_baseline_lgbm_test.parquet
pr_45_pgm_fat_jed_hh_baseline_lgbm_f506.parquet
1 fat_jed_hh_conflictlong_lgbm
pr_45_pgm_fat_jed_hh_conflictlong_lgbm_calib.parquet
pr_45_pgm_fat_jed_hh_conflictlong_lgbm_test.parquet
pr_45_pgm_fat_jed_hh_conflictlong_lgbm_f506.parquet
2 fat_jed_hh_conflictlong_hurdle_lgbm
pr_45_pgm_fat_jed_hh_conflictlong_hurdle_lgbm_calib.parquet
pr_45_pgm_fat_jed_hh_conflictlong_hurdle_lgbm_test.parquet
pr_45_pgm_fat_jed_hh_conflictlong_hurdle_lgbm_f506.parquet
3 fat_jed_hh_drought_hurdle_lgbm
pr_45_pgm_fat_jed_hh_drought_hurdle_lgbm_calib.parquet
pr_45_pgm_fat_jed_hh_drought_hurdle_lgbm_test.parquet
pr_45_pgm_fat_jed_hh_drought_hurdle_lgbm_f506.parquet
4 fat_jed_hh_drought_lgbm
pr_45_pgm_fat_jed_hh_drought_lgbm_calib.parquet
pr_45_pgm_fat_jed_hh_drought_lgbm_test.parquet
pr_45_pgm_fat_jed_hh_drought_lgbm_f506.parquet
5 fat_jed_hh_natsoc_hurdle_lgbm
pr_45_pgm_fat_jed_hh_natsoc_hurdle_lgbm_calib.parquet
pr_45_pgm_fat_jed_hh_natsoc_hurdle_lgbm_test.parqu

In [25]:
EnsembleList = [] # Separate list of dictionaries for ensembles!

ensemble = {
    'modelname':            'cm_calibrated_pgm_ensemble',
    'algorithm':            [],
    'depvar':               depvar,
    'data_train':           [],
    'Algorithm_text':       '',
    'calibration_gams':     [],
    'future_df_calibrated': [],
}
EnsembleList.append(ensemble)

In [32]:
cm_ensemble=ViewsMetadata().with_name('cm_genetic_ensemble_f'+str(EndOfHistory)).fetch()

In [33]:
calib_run_id=int(cm_ensemble['runs_id'].values[0])

In [35]:
cm_predictions_calib = pd.DataFrame.forecasts.read_store(run=calib_run_id, name='cm_ensemble_genetic_calib')
cm_predictions_test = pd.DataFrame.forecasts.read_store(run=calib_run_id, name='cm_ensemble_genetic_test')

pr_33_cm_ensemble_genetic_calib.parquet
pr_33_cm_ensemble_genetic_test.parquet


In [27]:
stepcols=['step_pred_' + str(step) for step in steps]

In [28]:
n_models = len(ModelList)

targetcalib=ModelList[0]['predictions_calib_df'][depvar]
targettest=ModelList[0]['predictions_test_df'][depvar]

valscalib=ModelList[0]['predictions_calib_df'][stepcols].values.copy()
valstest=ModelList[0]['predictions_test_df'][stepcols].values.copy()

trimmed_calib=ModelList[0]['predictions_calib_df'][stepcols].copy()
index_calib=trimmed_calib.index
columns_calib=trimmed_calib.columns

trimmed_test=ModelList[0]['predictions_test_df'][stepcols].copy()
index_test=trimmed_test.index
columns_test=trimmed_test.columns

for model in ModelList[1:]:
    print('adding',model['modelname'])

    valscalib+=model['predictions_calib_df'][stepcols].values.copy()
    valstest+=model['predictions_test_df'][stepcols].values.copy()

    valscalib/=n_models
    valstest/=n_models

    ensemble['calib_df_calibrated']=pd.DataFrame(data=valscalib, index=index_calib, columns=columns_calib)
    ensemble['test_df_calibrated']=pd.DataFrame(data=valstest, index=index_test, columns=columns_test)
    
df_pg_id_c_id=Ensembling.fetch_df_pg_id_c_id()

print()
    
for col in stepcols:
    
    print('calibrating step',col)

    thisstep=int(''.join([''+str(f) for f in filter(str.isdigit, col)]))
    thismonth = EndOfHistory + thisstep

    ensemble['calib_df_calibrated'][col]=Ensembling.calibrate_pg_with_c(ensemble['calib_df_calibrated'],cm_predictions_calib,col,df_pg_id_c_id=df_pg_id_c_id,log_feature=True,super_calibrate=False)

    ensemble['test_df_calibrated'][col]=Ensembling.calibrate_pg_with_c(ensemble['test_df_calibrated'],cm_predictions_test,col,df_pg_id_c_id=df_pg_id_c_id,log_feature=True,super_calibrate=False)
    

adding fat_jed_hh_conflictlong_lgbm
adding fat_jed_hh_conflictlong_hurdle_lgbm
adding fat_jed_hh_drought_hurdle_lgbm
adding fat_jed_hh_drought_lgbm
adding fat_jed_hh_natsoc_hurdle_lgbm
adding fat_jed_hh_natsoc_lgbm
adding fat_jed_hh_broad_hurdle_lgbm
adding fat_jed_hh_broad_lgbm
adding fat_jed_pv_conf_hist_pgm
adding fat_tree_lags_d_1_d_2_hur_36
adding fat_sptime_dist_nu1_10_001_hur_36
 .    
calibrating step step_pred_1
calibrating step step_pred_2
calibrating step step_pred_3
calibrating step step_pred_4
calibrating step step_pred_5
calibrating step step_pred_6
calibrating step step_pred_7
calibrating step step_pred_8
calibrating step step_pred_9
calibrating step step_pred_10
calibrating step step_pred_11
calibrating step step_pred_12
calibrating step step_pred_13
calibrating step step_pred_14
calibrating step step_pred_15
calibrating step step_pred_16
calibrating step step_pred_17
calibrating step step_pred_18
calibrating step step_pred_19
calibrating step step_pred_20
calibrating s

In [29]:
ensemble['calib_df_calibrated'][depvar]=targetcalib
ensemble['test_df_calibrated'][depvar]=targettest

In [30]:
# Save ensemble predictions
predstore_calib = level +  '_' + ensemble['modelname'] + '_calib'
ensemble['calib_df_calibrated'].forecasts.set_run(run_id)
ensemble['calib_df_calibrated'].forecasts.to_store(name=predstore_calib, overwrite = True)
predstore_test = level +  '_' + ensemble['modelname'] + '_test'
ensemble['test_df_calibrated'].forecasts.set_run(run_id)
ensemble['test_df_calibrated'].forecasts.to_store(name=predstore_test, overwrite = True)